In [1]:
library(mediation)

Loading required package: MASS
Loading required package: Matrix
Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.0



# Mediation analysis BMI -> HOMA_IR -> Brain activity in extracted ROIs

In [ ]:
ROI=read.table('/dagher/dagher11/filip/Downloads/ROIs_extracted_orange(1).csv',sep=',',header=TRUE)

Stats=matrix(nrow=12,ncol=10) # CT variables
index=1

med.fit <- lm(scale(HOMA_IR) ~
                scale(BMI) +
                scale(Age) +
                as.factor(Sex), data=ROI)

ROI_list=c('ACC','Amygdala','Caudate','Hippocampus','Insula','NAcc','Orbitofrontal','Pallidum','Piriform','Putamen','VmPFC','VTA')

for (i in c(1:12)) { #All ROIs
  
  out.fit <- lm(scale(ROI[grep(ROI_list[i],colnames(ROI))]) ~ 
                  scale(HOMA_IR) +
                  scale(BMI) +
                  scale(Age) +
                  as.factor(Sex), data=ROI)
  
  med.out <- mediate(med.fit, out.fit, treat='scale(BMI)',
                     mediator='scale(HOMA_IR)',
                     robustSE=TRUE,
                     sims=1000, data=ROI)
  
  med2dep <- summary(lm(scale(ROI[grep(ROI_list[i],colnames(ROI))]) ~ 
                          scale(HOMA_IR) +
                          scale(Age) +
                          as.factor(Sex), data=ROI)) 
  
  
  Stats[index,][1]=ROI_list[i]
  Stats[index,][2]=med.out$d0 # ACME estimate
  Stats[index,][3]=med.out$d0.p # ACME p-value
  Stats[index,][4]=med.out$z0 # ADE e
  Stats[index,][5]=med.out$z0.p # ADE p
  Stats[index,][6]=med.out$tau.coef # Total e
  Stats[index,][7]=med.out$tau.p # Total p
  Stats[index,][8]=med.out$n0 # Proportion mediated e
  Stats[index,][9]=med.out$n0.p # Proportion mediated p
  Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
  
  
  index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('ROI','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

  

In [ ]:
Stats_adj